In [ ]:
from datetime import datetime
import pandas as pd
import requests
import logging
import sys

In [ ]:
formatter = logging.Formatter("%(asctime)s [%(levelname)s] %(message)s")
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
handler.setFormatter(formatter)
logger = logging.getLogger()
logger.handlers.clear()
logger.setLevel(logging.INFO)
logger.addHandler(handler)

# download warrant info from 群益權民最大網
date = datetime.now().strftime('%Y-%m-%d')
url = f'https://iwarrant.capital.com.tw/wdataV2/canonical/capital-newvol/%E6%AC%8A%E8%AD%89%E9%81%94%E4%BA%BA%E5%AF%B6%E5%85%B8_NEWVOL_{date}.xls'
logger.info(url)
outputPath = ''
fileName = f'warrant_raw_{date}'
extension = '.xls'
# try:
resp = requests.get(url)
if resp.status_code == 200:
    # except requests.exceptions.HTTPError as e:
    #     raise SystemExit(e)


    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(outputPath + fileName + extension, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)

    # extract information to new CSV file
    pdf = pd.read_excel(outputPath + fileName + extension, sheet_name='summary', skiprows=4)
    colNames = ["權證代碼", "權證名稱", "發行券商", "權證價格", "權證漲跌", "權證漲跌幅", "權證成交量", "權證買價", "權證賣價", "權證買賣價差", "溢價比率", "價內價外", "理論價格", "隱含波動率", "有效槓桿", "剩餘天數", "最新行使比例", "標的代碼", "標的名稱", "標的價格", "標的漲跌", "標的漲跌幅", "最新履約價", "最新界限價", "標的20日波動率", "標的60日波動率", "標的120日波動率", "權證DELTA", "權證GAMMA", "權證VEGA", "權證THETA", "內含價值", "時間價值", "流通在外估計張數", "流通在外增減張數", "上市日期", "到期日期", "最新發行量", "權證發行價", "認購認售"]
    pdf.columns = colNames
    extension = '.csv'
    pdf.to_csv(outputPath + fileName.replace('_raw', '') + extension, index=False)
else:
    logger.error('status code:' + str(resp.status_code))